# RNN GRU Explainer

The goal is to walk through RNNs (recurrent neural networks), there are 2 flavors: GRUs and LSTMs. We'll focus on Gated Recurrent Unit (GRUs). RNN's have a unique structure where they process data sequentially which enables them to take in both the current and a prior state, making them great for sequential data like time series or streamed data.  The core unit of the RNN takes in both the data you want to use to make an inference along with any previous context. While this is great there are a few dangers of the RNN: primarily that improper setup and training can lead to vanishing/exploding gradients and their sequential nature can consume a lot of resources. As you go through this notebook you can hopefully see why this would be the case. 

To help display how the RNN works, we'll use the first sentence from the [linear algebra wiki page](https://en.wikipedia.org/wiki/Linear_algebra) and [lu decomposition wiki page](https://en.wikipedia.org/wiki/LU_decomposition) as the topic is fitting and it shows us some non-standard patterns.  We'll take on a more simple task to many other notebooks in this repository where we have a few sentences of text and want to predict some other text, in this case the next token. 

## Text Prep/Tokenization

We'll start with a common preprocessing step of tokenizing the data.  This converts the string text into an array of numbers that can be used during the training loop.  I've built a very subtle byte-pair encoding that has each unique character that appears and the top 5 merges. This keeps our vocab size small and manageable for this example. Typically the vocab size is in the 100K+ range. A great library for this is `tiktoken`. Tokenization simply finds the longest pattern of characters that's in common with what was trained and replaces it with an integer that represents it.  This way we turn the text into a numeric array to simplify computing. import torch
from collections import Counter

In [1]:
import torch
from collections import Counter

In [2]:
class SimpleBPETokenizer:
    def __init__(self, num_merges=5, eot_token='<|endoftext|>'):
        self.num_merges = num_merges
        self.eot_token = eot_token
        self.eot_id = None
        self.merges = []
        self.pair_ranks = {}
        self.vocab = {}
        self.id_to_token = {}

    def _add_token(self, tok):
        if tok in self.vocab:
            return self.vocab[tok]
        i = len(self.vocab)
        self.vocab[tok] = i
        self.id_to_token[i] = tok
        return i

    def _get_bigrams(self, seq):
        for i in range(len(seq) - 1):
            yield (seq[i], seq[i + 1])

    def _merge_once(self, seq, pair):
        a, b = pair
        out = []
        i = 0
        while i < len(seq):
            if i < len(seq) - 1 and seq[i] == a and seq[i + 1] == b:
                out.append(a + b)
                i += 2
            else:
                out.append(seq[i])
                i += 1
        return out

    def train(self, corpus):
        # corpus: list[str]
        text = ''.join(corpus).lower()
        seq = list(text)
        merges = []
        for _ in range(self.num_merges):
            counts = Counter(self._get_bigrams(seq))
            if not counts: break
            best_pair, _ = counts.most_common(1)[0]
            merges.append(best_pair)
            seq = self._merge_once(seq, best_pair)
        self.merges = merges
        self.pair_ranks = {p: i for i, p in enumerate(self.merges)}

        self.vocab = {}
        self.id_to_token = {}
        for ch in sorted(set(text)):
            self._add_token(ch)
        for a, b in self.merges:
            self._add_token(a + b)
        self.eot_id = self._add_token(self.eot_token)

    def encode(self, text, force_last_eot=True):
        # treat literal eot marker as special; remove it from content
        if self.eot_token in text:
            text = text.replace(self.eot_token, '')
        seq = list(text)

        # make sure all seen base chars exist
        for ch in set(seq):
            if ch not in self.vocab:
                self._add_token(ch)

        # greedy BPE using learned pair ranks
        if self.merges:
            while True:
                best_pair, best_rank = None, None
                for p in self._get_bigrams(seq):
                    r = self.pair_ranks.get(p)
                    if r is not None and (best_rank is None or r < best_rank):
                        best_pair, best_rank = p, r
                if best_pair is None:
                    break
                seq = self._merge_once(seq, best_pair)

        # ensure all tokens in seq exist in vocab (e.g., if new chars appeared)
        for tok in seq:
            if tok not in self.vocab:
                self._add_token(tok)

        ids = [self.vocab[tok] for tok in seq]

        # FORCE: append EOT id if not already last
        if force_last_eot:
            if not ids or ids[-1] != self.eot_id:
                ids.append(self.eot_id)

        return ids

    def decode(self, ids):
        # drop trailing EOT if present
        if ids and self.eot_id is not None and ids[-1] == self.eot_id:
            ids = ids[:-1]
        toks = [self.id_to_token[i] for i in ids]
        return ''.join(toks)


In [3]:
raw_example_1 = r'''Linear algebra is central to almost all areas of mathematics. For instance, linear algebra is fundamental in modern presentations of geometry, including for defining basic objects such as lines, planes and rotations. Also, functional analysis, a branch of mathematical analysis, may be viewed as the application of linear algebra to function spaces.'''
raw_example_2 = r'''In numerical analysis and linear algebra, lower–upper (LU) decomposition or factorization factors a matrix as the product of a lower triangular matrix and an upper triangular matrix (see matrix multiplication and matrix decomposition).'''


In [4]:
tok = SimpleBPETokenizer(num_merges=5)
tok.train([raw_example_1,raw_example_2])
tok.merges

[(' ', 'a'), ('a', 't'), ('i', 'n'), (' ', 'm'), ('i', 'o')]

In [5]:
tok.vocab

{' ': 0,
 '(': 1,
 ')': 2,
 ',': 3,
 '.': 4,
 'a': 5,
 'b': 6,
 'c': 7,
 'd': 8,
 'e': 9,
 'f': 10,
 'g': 11,
 'h': 12,
 'i': 13,
 'j': 14,
 'l': 15,
 'm': 16,
 'n': 17,
 'o': 18,
 'p': 19,
 'r': 20,
 's': 21,
 't': 22,
 'u': 23,
 'v': 24,
 'w': 25,
 'x': 26,
 'y': 27,
 'z': 28,
 '–': 29,
 ' a': 30,
 'at': 31,
 'in': 32,
 ' m': 33,
 'io': 34,
 '<|endoftext|>': 35}

In [6]:
vocab_size = len(tok.vocab)
vocab_size

36

In [7]:
eot = tok.eot_id
tokens = []
for example in [raw_example_1, raw_example_2]:
    tokens.extend([eot])
    tokens.extend(tok.encode(example.lower()))
all_tokens = torch.tensor(tokens, dtype=torch.long)
all_tokens

tensor([35, 15, 32,  9,  5, 20, 30, 15, 11,  9,  6, 20,  5,  0, 13, 21,  0,  7,
         9, 17, 22, 20,  5, 15,  0, 22, 18, 30, 15, 16, 18, 21, 22, 30, 15, 15,
        30, 20,  9,  5, 21,  0, 18, 10, 33, 31, 12,  9, 16, 31, 13,  7, 21,  4,
         0, 10, 18, 20,  0, 32, 21, 22,  5, 17,  7,  9,  3,  0, 15, 32,  9,  5,
        20, 30, 15, 11,  9,  6, 20,  5,  0, 13, 21,  0, 10, 23, 17,  8,  5, 16,
         9, 17, 22,  5, 15,  0, 32, 33, 18,  8,  9, 20, 17,  0, 19, 20,  9, 21,
         9, 17, 22, 31, 34, 17, 21,  0, 18, 10,  0, 11,  9, 18, 16,  9, 22, 20,
        27,  3,  0, 32,  7, 15, 23,  8, 32, 11,  0, 10, 18, 20,  0,  8,  9, 10,
        32, 32, 11,  0,  6,  5, 21, 13,  7,  0, 18,  6, 14,  9,  7, 22, 21,  0,
        21, 23,  7, 12, 30, 21,  0, 15, 32,  9, 21,  3,  0, 19, 15,  5, 17,  9,
        21, 30, 17,  8,  0, 20, 18, 22, 31, 34, 17, 21,  4, 30, 15, 21, 18,  3,
         0, 10, 23, 17,  7, 22, 34, 17,  5, 15, 30, 17,  5, 15, 27, 21, 13, 21,
         3, 30,  0,  6, 20,  5, 17,  7, 

# Modeling

A machine learning model forward pass now uses the tokenization information, runs several layers of linear algebra on it, and then "predicts" the next token. When it is noisy (like you will see in this example), this process results in gibberish.  The training process changes the noise to pattern during the "backward pass" as you'll see.    We'll show 3 steps that are focused on training:
1. **Data Loading** `x, y = train_loader.next_batch()` - this step pulls from the raw data enough tokens to complete a forward and backward pass.  If the model is inference only, this step is replaced with taking in the inference input and preparing it similarly as the forward pass.
2. **Forward Pass** `logits, loss = model(x, y)` - using the data and the model architecture to predict the next token. When training we also compare against the expected to get loss, but in infrerence, we use the logits to complete the inference task.
3. **Back Propagation, aka Backward Pass & Training** `loss.backward(); optimizer.step()` - using differentials to understand what parameters most impact the forward pass' impact on its prediction, comparing that against what is actually right based on the data loading step, and then making very minor adjustments to the impactful parameters with the hope it improves future predictions.

The we'll show a final **Forward Pass** with the updated weights we did in #3. 

## Data Loading

To start, we need to get enough data to run the forward and backward passes.  Since our total dataset is likely too big to hold all at once in real practice, we would read just enough file information into memory so that we can run the passes, leaving memory and compute to be used on the passes instead of static data holding. 
To start, we have to identify the batch size and the model context length to determine how much data we need.  Consequently, these dimensions also form 2 of the 3 dimensions in the initial matrix.
- **Batch Size (B)** - This is the number of examples you'll train on in a single pass. 
- **Context Length (T)** - This is the max number of tokens that a model can use in a single pass to generate the next token. If an example is below this length, it can be padded.
  
*Ideally both B and T are multiples of 2 to work nicely with chip architecture. This is a common theme across the board*

In [8]:
B = 2 # Batch
T = 8 # context length

To start, we need to pull from our long raw_token list enough tokens for the forward pass. To be able to satisfy training `B` Batches `T` Context length, we need to pull out `B*T` tokens to slide the context window across the examples enough to satisfy the batch size.  Since the training will attempt to predict the last token given the previous tokens in context, we also need 1 more token at the end so that the last training example in the last batch can have the next token to validate against. 

In [9]:
current_position = 0
tok_for_training = all_tokens[current_position:current_position + B*T +1 ]
tok_for_training

tensor([35, 15, 32,  9,  5, 20, 30, 15, 11,  9,  6, 20,  5,  0, 13, 21,  0])

Now that we have our initial tokens to train on, we now need to convert it to a matrix that's ready for training. In this step we'll need to create our batches and setup two different arrays: 1/ the input, `x`, tokens that will result in 2/ the output `y` tokens. To create each example in the batch, every `T` tokens will be placed into its own row. 

Recall that training takes in a string of tokens the length of the context and then predicts the next token. Recall that when we extracted `tok_for_training` we added 1 extra token so that we can evaluate the prediction for the last example. Because of this, the input, `x`, will be all of the tokens up to the second to last element `[:-1]`.  

It might be natural to think the output `y` would then just be the last token.But this is actually wasting valuable training loops.  Yes, there is the example that fills the context `T`, but we also have enough tokens in `tok_for_training` where any context length of `n` where `n<T` can also be used for inference since we have the `n+1` token available.  You can think of the following example:

sentence: `Hi I am learning`. This sentence contains the following "next tokens" that can be learned:
1. x: Hi I am  | y: learning
2. x: Hi I     | y: am
3. x: Hi       | y: I

Because we have this triangle to create, our `y` can be much larger.  We can start with the second token and, go all the way to the last element we added for the last example `[1:'`.   


We will now put this together and do the following:
1. Extract the input `x` and then split it into an example for each batch `B`
2. Extract the output `y` and then split it into an example for each batch `B`

*Note: View can take `-1` which allows the matrix to infer the dimension so we do not need to pass in `T`, but given how many matrices we'll work with we want to make sure we're controlling the dimensions or erroring out if they do not match our expectations.*

In [10]:
x=tok_for_training[:-1].view(B, T)
x

tensor([[35, 15, 32,  9,  5, 20, 30, 15],
        [11,  9,  6, 20,  5,  0, 13, 21]])

In [11]:
y=tok_for_training[1:].view(B, T)
y

tensor([[15, 32,  9,  5, 20, 30, 15, 11],
        [ 9,  6, 20,  5,  0, 13, 21,  0]])

## Forward pass

<img src="explainer_screenshots/gru/full_network_gru.png" width="200">

During training, the forward pass takes a string of tokens in and predicts the likelihood that each output token is the next "n" tokens.  This step as we'll look at it is focused on training where we'll pass in the input `x`, carry that input through the layers, and generate a matrix of the probability of each token being the next one, something we call `logits`. During the forward pass, since this is an RNN, we will actually pass each example through recursively based on the token length of available. For example, if we have 3 tokens in the example and we are deriving the 4th, we would hit our GRU block 3 times through:
1. First with the embedding for the token `[0]` and no previous token
2. Then with the embedding for the token `[1]` and the output of the GRU unit's processing of token `[0]` as the previous $H_{T-1}$
3. Finally with the embedding for the token `[2]` and the output of the GRU unit's processing of token `[1]` as the previous $H_{T-1}$

At the end of the forward pass at the end we then compare the probability to the actual next token in `y` and calculate `loss` based on the difference. 

*Note that we will do some layer initialization to simplify following along.  In reality layers are often initialized to normal distribution with some adjustments made for parameter sizes to keep the weights properly noisy.  We will not cover initialization in this series*

We first rederive the batch size and context size based on the input to improve flexibility.

In [12]:
import torch.nn as nn

In [13]:
B_batch, T_context = x.size()
B_batch, T_context

(2, 8)

In [14]:
n_embd = 4 # level of embedding of input tokens
n_embd, vocab_size

(4, 36)

### Input Layer

<img src="explainer_screenshots/gru/input_layer.png" width="200">

We'll first create an initial **embedding layer** for our input tokens.  Since the RNN iteratively processes input data, position embedding info is not needed, though, it can always be added to give the model more depth. The embedding weights are `vocab_size X n_embd` and simply store weights that correspond with each token.  The more embedding layers added the more complex data the model can learn. After the embedding layer we'll then perform dropout. The **dropout layer** simply applies 0's  randomly up to the defined level percentage and normalizes the remaining probabilities in the row. Since RNNs recycle the same parameters across timesteps, features can co-adapt and overfit quickly. Variational dropout breaks these temporal co-adaptations and adds needed noise where there’s little implicit regularization. Other model architectures like transformers have strong built-in stabilizers (LayerNorm, residuals, attention) and massive pretraining, so they generally need far less dropout. 



**Embedding** 

To start we'll initialize our embeddings with a weight of 1.000 so that all inputs are equally weighted. We'll also set our embedding dimension to 4 to allow for some levels of complexity. Since our initiation weights are equal, we expect that the output of embedding will for now be equal

In [15]:
wte = nn.Embedding(vocab_size, n_embd)
torch.nn.init.ones_(wte.weight)
wte.weight

Parameter containing:
tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]], requires_grad=True)

In [16]:
x = wte(x)
x.shape, x

(torch.Size([2, 8, 4]),
 tensor([[[1., 1., 1., 1.],
          [1., 1., 1., 1.],
          [1., 1., 1., 1.],
          [1., 1., 1., 1.],
          [1., 1., 1., 1.],
          [1., 1., 1., 1.],
          [1., 1., 1., 1.],
          [1., 1., 1., 1.]],
 
         [[1., 1., 1., 1.],
          [1., 1., 1., 1.],
          [1., 1., 1., 1.],
          [1., 1., 1., 1.],
          [1., 1., 1., 1.],
          [1., 1., 1., 1.],
          [1., 1., 1., 1.],
          [1., 1., 1., 1.]]], grad_fn=<EmbeddingBackward0>))

### Dropout

Now, before our Recurrent unit, we'll perform dropout. Dropout will randomly zero out any value effectively removing that specific node from impacting prediction. Since this is Bernoulli based dropout, in addition to zeroing out weights the surviving entries are scaled by $1/(1-p)$. During training this helps with generalization and fights fixation. You can quickly see the dropout's impact on the embeddings. 

In [17]:
dropout = nn.Dropout(0.1)
dropout

Dropout(p=0.1, inplace=False)

In [18]:
x = dropout(x)
x

tensor([[[1.1111, 1.1111, 1.1111, 1.1111],
         [1.1111, 0.0000, 1.1111, 1.1111],
         [1.1111, 1.1111, 0.0000, 1.1111],
         [1.1111, 1.1111, 1.1111, 1.1111],
         [0.0000, 1.1111, 1.1111, 1.1111],
         [0.0000, 1.1111, 0.0000, 1.1111],
         [0.0000, 1.1111, 1.1111, 1.1111],
         [1.1111, 1.1111, 1.1111, 1.1111]],

        [[1.1111, 0.0000, 1.1111, 1.1111],
         [1.1111, 1.1111, 1.1111, 1.1111],
         [1.1111, 0.0000, 1.1111, 1.1111],
         [1.1111, 1.1111, 1.1111, 1.1111],
         [1.1111, 0.0000, 1.1111, 1.1111],
         [1.1111, 1.1111, 1.1111, 0.0000],
         [1.1111, 1.1111, 0.0000, 1.1111],
         [1.1111, 1.1111, 1.1111, 1.1111]]], grad_fn=<MulBackward0>)

### Recurrent Unit - GRU

<img src="explainer_screenshots/gru/gru_details.png" width="600">

At a high level, you'll see that our GRU acts as our recurrent unit.  This unit does not take in the whole context `X` but rather iteratively takes in a specific time point $X_t$, calculates its weights $H_t$. If there are more time points, for us meaning more tokens in the string, the unit then takes in that weight, represented as $H_{prev}$, and the next time point, represented again as $X_t$ and reruns the unit, until all time points are exhausted. 

To manage the impact of the incoming state and the long term memory, GRUs use two gates: the reset gate and the update gate. Both of these gates push their values to lie between $[0,1]$ through the use of a sigmoid activation. 

Mentally the **Update Gate**  $Z$ controls how much of the new state is a copy of the old vs being new. We achieve this by applying the linearization $AX+WH+b$ to both the previous weights $H_{prev}$ and the incoming weights $X_t$. This results in a gate that acts as a learnable proportion that controls what proportion of the previous chain is through $Z \odot H_{prev}$ and the remaining proportion is applied to the candidate hidden state $(1 - Z) \odot \tilde{H}_t$ as calculated through the *Reset Gate*. 

The **Reset Gate** $R$ is our control on how much of the previous state to remember in the candidate hidden state.  We similarly calculate this by applying the linearization $AX+WH+b$ to both the previous weights $H_{prev}$ and the incoming weights $X_t$.  

The **Candidate Hidden State** $\tilde{H}_t$ is our representation of the impact of the timepoint's specific context $X_t$  given the previous context $H_{prev}$.  The amount of previous context is decided by the *reset gate* $R$ that's learned. We again use linearization to these two inputs $AX+WR\odot H+b$ and then wrap it in a $tanh$ to pull the values to $[-1,1]$ to limit the candidate hidden state’s magnitude and keeps activations zero-centered, improving stability and optimization while the gates handle long-range impact assignment.

You'll notice that we use the $\odot$ representing the Hadamard product, or the element wise multiple, instead of the dot product.  In RNNs we use this because the gates are meant to be per-feature “valves.” The update uses elementwise products to preserve the shape and provide an element wise identity path for gradients. A dot product would either collapse to a scalar (vector·vector) or mix features via a full matrix multiply, which changes the operation from gating to another affine transform, adds parameters, couples coordinates, and weakens the stable skip-like behavior that helps GRUs retain memory and avoid vanishing gradients.

All together this results in the following:

$$
\begin{align}
Z &= \sigma\!\left(X_t W_{xz} + H_{prev} W_{hz} + b_z\right)\ \ \ \ \text{Update Gate},\\
R &= \sigma\!\left(X_t W_{xr} + H_{prev} W_{hr} + b_r\right)\ \ \ \ \text{Reset Gate},\\
\tilde{H}_t &= \tanh\!\left(X_t W_{xh} + R \odot \left(H_{prev}W_{hh}\right) + b_h \right),\\
H_t &= Z \odot H_{prev} + \left(1-Z_t\right)\odot \tilde{H}_t .
\end{align}
$$

Mathematically this gives us a nice simple set of formulas, but we need to actually write this as a program. If we look closely we have 3 sets of weights acting:
* the input $X_t$: $\{W_{xz}, W_{xr}, W_{xh}\}$. *Notice the subscript is "x\<gate or state>"*
* the previous $H_{prev}$: $\{W_{hz}, W_{hr}, W_{hh}\}$.

To simplify this weight we'll start by creating a linear later that has 3x the hidden layers, separate it, and then complete the rest of the calculations. This allows us to gain efficiencies in learning and improve our code readability.  The algorithmic representation then looks as follows:

```
xz, xr, xh = x2g(x_t).split(hidden_size, dim=-1)
hz, hr, hh = h2g(h_prev).split(hidden_size, dim=-1)
z = sigma(xz + hz)
r = sigma(xr + hr)
r_hh = r * hh
h_can = tanh(xh + r_hh})
h_t &= (1 - z)*h_can + z*h_prev.
```
**Starting Example** While GRUs are iterative, we'll first walk through 1 pass of the GRU. After that we'll show the loop for the rest of the context window/examples. This will help us understand what is happening inside a pass and then how the impact of the weights adds up over the context length. To keep things simple we'll set the `hidden_size` to 4. Let's start by initiating our weights.  To visualize the impact of each set of weights, I'll initialize the current context and the previous context weights to two different values.

In [19]:
hidden_size = 4

In [20]:
x2g = nn.Linear(n_embd, 3 * hidden_size, bias=True)
torch.nn.init.constant_(x2g.weight, 0.500)
torch.nn.init.zeros_(x2g.bias)
x2g.weight.size(), x2g.weight, x2g.bias.size(), x2g.bias

(torch.Size([12, 4]),
 Parameter containing:
 tensor([[0.5000, 0.5000, 0.5000, 0.5000],
         [0.5000, 0.5000, 0.5000, 0.5000],
         [0.5000, 0.5000, 0.5000, 0.5000],
         [0.5000, 0.5000, 0.5000, 0.5000],
         [0.5000, 0.5000, 0.5000, 0.5000],
         [0.5000, 0.5000, 0.5000, 0.5000],
         [0.5000, 0.5000, 0.5000, 0.5000],
         [0.5000, 0.5000, 0.5000, 0.5000],
         [0.5000, 0.5000, 0.5000, 0.5000],
         [0.5000, 0.5000, 0.5000, 0.5000],
         [0.5000, 0.5000, 0.5000, 0.5000],
         [0.5000, 0.5000, 0.5000, 0.5000]], requires_grad=True),
 torch.Size([12]),
 Parameter containing:
 tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], requires_grad=True))

In [21]:
h2g = nn.Linear(hidden_size, 3 * hidden_size, bias=True)
torch.nn.init.constant_(h2g.weight, 0.250)
torch.nn.init.zeros_(h2g.bias)
h2g.weight.size(), h2g.weight, h2g.bias.size(), h2g.bias

(torch.Size([12, 4]),
 Parameter containing:
 tensor([[0.2500, 0.2500, 0.2500, 0.2500],
         [0.2500, 0.2500, 0.2500, 0.2500],
         [0.2500, 0.2500, 0.2500, 0.2500],
         [0.2500, 0.2500, 0.2500, 0.2500],
         [0.2500, 0.2500, 0.2500, 0.2500],
         [0.2500, 0.2500, 0.2500, 0.2500],
         [0.2500, 0.2500, 0.2500, 0.2500],
         [0.2500, 0.2500, 0.2500, 0.2500],
         [0.2500, 0.2500, 0.2500, 0.2500],
         [0.2500, 0.2500, 0.2500, 0.2500],
         [0.2500, 0.2500, 0.2500, 0.2500],
         [0.2500, 0.2500, 0.2500, 0.2500]], requires_grad=True),
 torch.Size([12]),
 Parameter containing:
 tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], requires_grad=True))

#### GRU - Frist Pass

##### GRU - Previous $H_{prev}$
Let's start first by setting up our previous state weights.  Since we are at 'time 0' in training, there are no previous states yet. Because of this we'll actually set our state to just 0's, meaning no tokens or weights.  We'll then pass this into the linear layer and split it up into our 3 different linear units. Notice that because our previous state is currently 0, the weights returned are all 0. In future recursions, these weights will no longer be 0. 

In [22]:
h_prev = torch.zeros(B_batch, hidden_size) 
h_prev

tensor([[0., 0., 0., 0.],
        [0., 0., 0., 0.]])

In [23]:
h_prev_w = h2g(h_prev)
hz, hr, hh = h_prev_w.split(hidden_size, dim=-1)

'hz', hz, hz.size(), 'hr',hr, hr.size(),'hh',hh, hh.size()

('hz',
 tensor([[0., 0., 0., 0.],
         [0., 0., 0., 0.]], grad_fn=<SplitBackward0>),
 torch.Size([2, 4]),
 'hr',
 tensor([[0., 0., 0., 0.],
         [0., 0., 0., 0.]], grad_fn=<SplitBackward0>),
 torch.Size([2, 4]),
 'hh',
 tensor([[0., 0., 0., 0.],
         [0., 0., 0., 0.]], grad_fn=<SplitBackward0>),
 torch.Size([2, 4]))

##### GRU - Incoming $X_t$
Next we need to linearize our incoming state. Currently X has all 8 examples for our context windows across the 2 batches. With GRU's we can only feed in 1 example at a time and, since we want sequential weights, 1 token at a time.  Luckily our examples are incremental (example 1 is token 1, example 2 is tokens 1 and 2) so we can iteratively calculate our GRU weights per example and use that collectively as the weight for each example.  But to start we'll need to extract just the example for the first time period `x_t` for each batch and calculate our weights.

Since our weights are 1/2 for each value of the cell, you can see that when we multiply that by the example each weight remains consistent. If dropout is observed, the values in each batch can be different, but they'll be the same across the example. 

In [24]:
x_t = x[:, 0, :]
x_t

tensor([[1.1111, 1.1111, 1.1111, 1.1111],
        [1.1111, 0.0000, 1.1111, 1.1111]], grad_fn=<SliceBackward0>)

In [25]:
x_t_w = x2g(x_t)
xz, xr, xh = x_t_w.split(hidden_size, dim=-1)

'xz', xz, xz.size(), 'xr',xr, xr.size(),'xh',xh, xh.size()

('xz',
 tensor([[2.2222, 2.2222, 2.2222, 2.2222],
         [1.6667, 1.6667, 1.6667, 1.6667]], grad_fn=<SplitBackward0>),
 torch.Size([2, 4]),
 'xr',
 tensor([[2.2222, 2.2222, 2.2222, 2.2222],
         [1.6667, 1.6667, 1.6667, 1.6667]], grad_fn=<SplitBackward0>),
 torch.Size([2, 4]),
 'xh',
 tensor([[2.2222, 2.2222, 2.2222, 2.2222],
         [1.6667, 1.6667, 1.6667, 1.6667]], grad_fn=<SplitBackward0>),
 torch.Size([2, 4]))

##### GRU - Update Gate $Z$
Recall that the update gate controls how much of the new state is a copy of the old vs being new. We derive the update gate using the following: 
$Z = \sigma\!\left(X_t W_{xz} + H_{prev} W_{hz} + b_z\right)$

This update gate path will become the learnable weights that will control what proportion of the previous chain maintained through the recursions. As a reminder the sigmoid function pulls the values to $[0,1]$ [source](https://docs.pytorch.org/docs/stable/generated/torch.sigmoid.html) using:

$\sigma(x) = \frac{1}{1 + e^{(-x)}}$

In [26]:
z = torch.sigmoid(xz + hz)
z.size(), z

(torch.Size([2, 4]),
 tensor([[0.9022, 0.9022, 0.9022, 0.9022],
         [0.8411, 0.8411, 0.8411, 0.8411]], grad_fn=<SigmoidBackward0>))

##### GRU - Reset Gate $R$
Recall that the reset gate controls how much of the previous state to remember in the candidate hidden state. We derive the update gate using the following: 


$R = \sigma\!\left(X_t W_{xr} + H_{prev} W_{hr} + b_r\right)$


This reset gate path will become the learnable weights that will control what proportion of the previous chain is forgotten when using the current context to predict the next token in the candidate hidden state.  We use the same sigmoid function to squeeze the values to $[0,1]$

You can see that since the network is untrained, the values are the same as the update gate. With training, these values will diverge. 

In [27]:
r = torch.sigmoid(xr + hr)
r.size(), r

(torch.Size([2, 4]),
 tensor([[0.9022, 0.9022, 0.9022, 0.9022],
         [0.8411, 0.8411, 0.8411, 0.8411]], grad_fn=<SigmoidBackward0>))

##### GRU - Candidate Hidden State $\tilde{H}_t$

Recall that the Candidate Hidden State is our representation of the impact of the timepoint's specific context $X_t$  given the previous context $H_{prev}$. The amount the previous context impacts is learned through the reset gate $R$.  We calculate it as follows:

$\tilde{H}_t = \tanh\!\left(X_t W_{xh} + R_t \odot \left(H_{prev}W_{hh}\right) + b_h \right),\\$

This calculation sums the weight of the timepoint specific context with the weight of the reset gate then use a $tanh$ layer to pull values to $[-1,1]$ with the goal of avoiding vanishing/exploding gradients. 

We'll start by first calculating the weight of the reset gate `r_hh` and then do the sum. You'll notice since our $H_{prev}$ was 0, our reset gate will be zeroed out leaving just the incoming weights.  In subsequent recursions this will no longer be the case. 

In [28]:
r_hh = r*hh
r_hh.size(), r_hh

(torch.Size([2, 4]),
 tensor([[0., 0., 0., 0.],
         [0., 0., 0., 0.]], grad_fn=<MulBackward0>))

In [29]:
h_cand = torch.tanh(xh + r_hh)
h_cand.size(), h_cand

(torch.Size([2, 4]),
 tensor([[0.9768, 0.9768, 0.9768, 0.9768],
         [0.9311, 0.9311, 0.9311, 0.9311]], grad_fn=<TanhBackward0>))

##### GRU - Recursion output
We now have to combine our candidate hidden state and our previous.  We've calculated the update gate $Z$ that will be the proportion of how much the previous and the candidate contribute.  Since we want to maintain the per-feature impacts we use the Hadamard product here between the reset gate (or 1-z) and the weights.  For this pass we'll actually calculate each portion separately then combine them to be our new $H_t$. 

We'll first start by calculating the weights of the candidate `(1-z)*h_cand` and then of the previous `z*h_prev`.  Since this is the first recursion, we will see that the values of the previous are zeroed out and all impact is coming from our current context, as expected. 

In [30]:
(1 - z)

tensor([[0.0978, 0.0978, 0.0978, 0.0978],
        [0.1589, 0.1589, 0.1589, 0.1589]], grad_fn=<RsubBackward1>)

In [31]:
h_cand_imp = (1 - z)*h_cand
h_cand_imp.size(), h_cand_imp

(torch.Size([2, 4]),
 tensor([[0.0955, 0.0955, 0.0955, 0.0955],
         [0.1479, 0.1479, 0.1479, 0.1479]], grad_fn=<MulBackward0>))

In [32]:
h_prev_imp = z*h_prev
h_prev_imp.size(), h_prev_imp

(torch.Size([2, 4]),
 tensor([[0., 0., 0., 0.],
         [0., 0., 0., 0.]], grad_fn=<MulBackward0>))

In [33]:
h_t = h_cand_imp + h_prev_imp
h_t.size(), h_t

(torch.Size([2, 4]),
 tensor([[0.0955, 0.0955, 0.0955, 0.0955],
         [0.1479, 0.1479, 0.1479, 0.1479]], grad_fn=<AddBackward0>))

#### GRU -  Managing examples in a batch
Now that we have the weights from our gated rerecurrent unit for our first time period `t=0`, since we have more context we actually have to recur through the rest of the context.  Before we do the next cycle recall that for training we want to take advantage of the fact that each of our batches has an example for each value up to our context length.  We want to train across inputs of various lengths and so, we'll store our current weights that signify an input of just a single token into our context.  As you'll see in our recursion, we'll store the weight at the end of each pass and build up an array of tensors for each example in the batch. to take to our output layer. 

To start we'll create an empty tensor of `(B x T x hidden_size)`. We'll then update the `[:,t,:]` entry with `h_t` linking back to each recursion. 

In [34]:
hs = x.new_empty(B_batch, T_context, hidden_size)
hs.size(), hs

(torch.Size([2, 8, 4]),
 tensor([[[0., 0., 0., 0.],
          [0., 0., 0., 0.],
          [0., 0., 0., 0.],
          [0., 0., 0., 0.],
          [0., 0., 0., 0.],
          [0., 0., 0., 0.],
          [0., 0., 0., 0.],
          [0., 0., 0., 0.]],
 
         [[0., 0., 0., 0.],
          [0., 0., 0., 0.],
          [0., 0., 0., 0.],
          [0., 0., 0., 0.],
          [0., 0., 0., 0.],
          [0., 0., 0., 0.],
          [0., 0., 0., 0.],
          [0., 0., 0., 0.]]]))

now we'll enter in our first entry at `t=0`

In [35]:
hs[:, 0, :] = h_t   
hs

tensor([[[0.0955, 0.0955, 0.0955, 0.0955],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000]],

        [[0.1479, 0.1479, 0.1479, 0.1479],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000]]], grad_fn=<CopySlices>)

#### GRU - Second pass 

Now that we've done the first pass through, let's see how the second pass changes things. The main difference is that in the second pass we'll use the previous pass' output `h_t` to become the new previous `h_prev`.  This will quickly begin to show the impact now on the update gate and reset gate as the `h_prev` is no longer 0 but our X weights are still consistent. We'll start by updating `h_prev` and extracting the next set of inputs for `t=1`.

One thing to notice is that despite having an input `h_prev`, the values remain consistent across examples since our initiation weights are uniform.  

In [36]:
h_prev = h_t
x_t = x[:, 1, :]

h_prev, x_t

(tensor([[0.0955, 0.0955, 0.0955, 0.0955],
         [0.1479, 0.1479, 0.1479, 0.1479]], grad_fn=<AddBackward0>),
 tensor([[1.1111, 0.0000, 1.1111, 1.1111],
         [1.1111, 1.1111, 1.1111, 1.1111]], grad_fn=<SliceBackward0>))

**GRU second pass - generating Weights**

In [37]:
h_prev_w = h2g(h_prev)
hz, hr, hh = h_prev_w.split(hidden_size, dim=-1)

'hz', hz, hz.size(), 'hr',hr, hr.size(),'hh',hh, hh.size()

('hz',
 tensor([[0.0955, 0.0955, 0.0955, 0.0955],
         [0.1479, 0.1479, 0.1479, 0.1479]], grad_fn=<SplitBackward0>),
 torch.Size([2, 4]),
 'hr',
 tensor([[0.0955, 0.0955, 0.0955, 0.0955],
         [0.1479, 0.1479, 0.1479, 0.1479]], grad_fn=<SplitBackward0>),
 torch.Size([2, 4]),
 'hh',
 tensor([[0.0955, 0.0955, 0.0955, 0.0955],
         [0.1479, 0.1479, 0.1479, 0.1479]], grad_fn=<SplitBackward0>),
 torch.Size([2, 4]))

In [38]:
x_t_w = x2g(x_t)
xz, xr, xh = x_t_w.split(hidden_size, dim=-1)

'xz', xz, xz.size(), 'xr',xr, xr.size(),'xh',xh, xh.size()

('xz',
 tensor([[1.6667, 1.6667, 1.6667, 1.6667],
         [2.2222, 2.2222, 2.2222, 2.2222]], grad_fn=<SplitBackward0>),
 torch.Size([2, 4]),
 'xr',
 tensor([[1.6667, 1.6667, 1.6667, 1.6667],
         [2.2222, 2.2222, 2.2222, 2.2222]], grad_fn=<SplitBackward0>),
 torch.Size([2, 4]),
 'xh',
 tensor([[1.6667, 1.6667, 1.6667, 1.6667],
         [2.2222, 2.2222, 2.2222, 2.2222]], grad_fn=<SplitBackward0>),
 torch.Size([2, 4]))

**GRU second pass - Update Gate**

In [39]:
z = torch.sigmoid(xz + hz)
z.size(), z

(torch.Size([2, 4]),
 tensor([[0.8535, 0.8535, 0.8535, 0.8535],
         [0.9145, 0.9145, 0.9145, 0.9145]], grad_fn=<SigmoidBackward0>))

**GRU second pass - Reset Gate**

In [40]:
r = torch.sigmoid(xr + hr)
r.size(), r

(torch.Size([2, 4]),
 tensor([[0.8535, 0.8535, 0.8535, 0.8535],
         [0.9145, 0.9145, 0.9145, 0.9145]], grad_fn=<SigmoidBackward0>))

**GRU second pass - Candidate hidden State**

In [41]:
r_hh = r*hh
r_hh.size(), r_hh

(torch.Size([2, 4]),
 tensor([[0.0815, 0.0815, 0.0815, 0.0815],
         [0.1353, 0.1353, 0.1353, 0.1353]], grad_fn=<MulBackward0>))

In [42]:
h_cand = torch.tanh(xh + r_hh)
h_cand.size(), h_cand

(torch.Size([2, 4]),
 tensor([[0.9412, 0.9412, 0.9412, 0.9412],
         [0.9822, 0.9822, 0.9822, 0.9822]], grad_fn=<TanhBackward0>))

**GRU second pass - Recursion Output**

In [43]:
h_cand_imp = (1 - z)*h_cand
h_cand_imp.size(), h_cand_imp

(torch.Size([2, 4]),
 tensor([[0.1379, 0.1379, 0.1379, 0.1379],
         [0.0840, 0.0840, 0.0840, 0.0840]], grad_fn=<MulBackward0>))

In [44]:
h_prev_imp = z*h_prev
h_prev_imp.size(), h_prev_imp

(torch.Size([2, 4]),
 tensor([[0.0815, 0.0815, 0.0815, 0.0815],
         [0.1353, 0.1353, 0.1353, 0.1353]], grad_fn=<MulBackward0>))

In [45]:
h_t = h_cand_imp + h_prev_imp
h_t.size(), h_t

(torch.Size([2, 4]),
 tensor([[0.2194, 0.2194, 0.2194, 0.2194],
         [0.2192, 0.2192, 0.2192, 0.2192]], grad_fn=<AddBackward0>))

**GRU second pass - Stashing Example Output**

In [46]:
hs[:, 1, :] = h_t   
hs

tensor([[[0.0955, 0.0955, 0.0955, 0.0955],
         [0.2194, 0.2194, 0.2194, 0.2194],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000]],

        [[0.1479, 0.1479, 0.1479, 0.1479],
         [0.2192, 0.2192, 0.2192, 0.2192],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000]]], grad_fn=<CopySlices>)

#### GRU - Recursion 
Now that we've seen how 2 passes through the GRU look like, let's write a loop to do the remaining passes.  The loop in this case will start at `t=2` and run the rest of the context length.  We'll capture the predicted weights into `hs` so that we'll have each example's weights and the link back for the backward pass during back propagation. 

In [47]:
for t in range(2,T_context):
    
    # set input and previous
    h_prev = h_t
    x_t = x[:, t, :]

    # calculate weights
    hz, hr, hh = h2g(h_prev).split(hidden_size, dim=-1)
    xz, xr, xh = x2g(x_t).split(hidden_size, dim=-1)

    # update gate
    z = torch.sigmoid(xz + hz)

    # reset gate
    r = torch.sigmoid(xr + hr)

    # candidate hidden state
    h_cand = torch.tanh(xh + (r * hh))

    # recursion output
    h_cand_imp = (1 - z) * h_cand
    h_prev_imp = z * h_prev
    h_t = h_cand_imp + h_prev_imp

    # save h_t
    hs[:, t, :] = h_t   

hs

tensor([[[0.0955, 0.0955, 0.0955, 0.0955],
         [0.2194, 0.2194, 0.2194, 0.2194],
         [0.3159, 0.3159, 0.3159, 0.3159],
         [0.3651, 0.3651, 0.3651, 0.3651],
         [0.4344, 0.4344, 0.4344, 0.4344],
         [0.5161, 0.5161, 0.5161, 0.5161],
         [0.5623, 0.5623, 0.5623, 0.5623],
         [0.5873, 0.5873, 0.5873, 0.5873]],

        [[0.1479, 0.1479, 0.1479, 0.1479],
         [0.2192, 0.2192, 0.2192, 0.2192],
         [0.3158, 0.3158, 0.3158, 0.3158],
         [0.3650, 0.3650, 0.3650, 0.3650],
         [0.4343, 0.4343, 0.4343, 0.4343],
         [0.4924, 0.4924, 0.4924, 0.4924],
         [0.5420, 0.5420, 0.5420, 0.5420],
         [0.5686, 0.5686, 0.5686, 0.5686]]], grad_fn=<CopySlices>)

Combine out recurring inputs into the 2 different batches

### Final Dropout 

After completing our recurrent unit, we'll perform another round of dropout. Dropout after the GRU regularizes the layer output / readout without corrupting the recurrent dynamics; injecting fresh noise inside the recurrence each step tends to destabilize long-range memory.  Often this step has “locked” dropout, meaning a single mask broadcast across time, which gives the same effect as per-step dropout with a fixed mask but avoids i.i.d. noise at every timestep.  We, however, will be doing default dropout that is not locked to simplify our code. 

As a reminder, we perform Bernoulli based dropout which both removes entries and adds p to the surviving entries. 

In [48]:
x = dropout(hs)
x.size(), x

(torch.Size([2, 8, 4]),
 tensor([[[0.1061, 0.1061, 0.1061, 0.1061],
          [0.2438, 0.2438, 0.2438, 0.2438],
          [0.3510, 0.3510, 0.0000, 0.3510],
          [0.4056, 0.0000, 0.4056, 0.4056],
          [0.4827, 0.4827, 0.4827, 0.0000],
          [0.5735, 0.5735, 0.5735, 0.5735],
          [0.6248, 0.6248, 0.6248, 0.6248],
          [0.6526, 0.6526, 0.6526, 0.0000]],
 
         [[0.1644, 0.1644, 0.1644, 0.1644],
          [0.2436, 0.2436, 0.2436, 0.2436],
          [0.3509, 0.3509, 0.3509, 0.3509],
          [0.4055, 0.4055, 0.4055, 0.4055],
          [0.4826, 0.4826, 0.4826, 0.4826],
          [0.5472, 0.5472, 0.5472, 0.0000],
          [0.6022, 0.6022, 0.6022, 0.6022],
          [0.6318, 0.6318, 0.6318, 0.6318]]], grad_fn=<MulBackward0>))

### Output Layers AKA Model Head.

The GRU unit is our recurrent unit for this example. In practice, this unit can be scaled by increasing the hidden_size dimension or increasing the complexity and flavors of the recurring unit. Once those layers are complete and we've done dropout normalization, during the forward pass we then start the output process that results in `logits` which is a representation of the probability of each token being the next token given the input.  

This layer is also known as the model **head**. This layer is called this because it is a small, task-specific module attached to a model’s shared backbone that maps hidden features to the final outputs.  In our example case, this is a linear layer mapping the backbone to vocab logits. The benefit of this structure is that you can use the shared hidden features and train different heads for different tasks without starting from scratch. An example would be a classifier head, or policy head in RL.

<img src="explainer_screenshots/gru/output_layer.png" width="200">

We'll use a similar head that we've used in other examples that focuses on predicting the next token. For our head we want to map to a predicted token which we'll look at as `logits`. In the process to generate `logits` we take the dropout output `x` of the GRU, then project, using a linear layer, to the vocabulary resulting in a `B, T, vocab_size` matrix known as `logits`.  

In training, the `logits` are then compared with `y` to see how close the  model is to predicting the correct next token. For inference, the `logits` are then used to drive sampling which is how the next token is then derived. 

Contrary to our transformer example, we won't be doing weight tying and instead use an initial linear layer.  We'll start initially by setting all of the values equal to 1 meaning they all have equal probability.  This will highlight just how impactful back-propagation and training is.  

In [49]:
lm_head = nn.Linear(hidden_size, vocab_size, bias=False)
torch.nn.init.ones_(lm_head.weight)
lm_head.weight.size(), lm_head.weight

(torch.Size([36, 4]),
 Parameter containing:
 tensor([[1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [

#### Output layer - LM Head aka logits
We now project `x` onto the vocabulary resulting in a `B X T X vocab_size` final array `logits`.  This output correlates with the probability of each output token given the input context.  The best way to read  this is:

(dimension 0) we have 2 batches B, 
(dimension 1) each batch has an example for each value between 1 and context length T 
(dimension 2) for each example we see the probability of each token in our vocabulary

Since our output head had consistent weights across all the dimensions, we fully expect that our logits will have equal values across all positions.  In practice this means that our model will have the same probability of a token output as the 'next token' regardless of the preceding text, meaning it's shit. Luckily back-propagation has a way of updating this so that with enough data and time the probabilities change. 

Note that if we wanted to run inference, after calculating the logits we'd run a softmax to sample a token out of the probability distribution. 

In [50]:
logits = lm_head(x)

logits.shape, logits

(torch.Size([2, 8, 36]),
 tensor([[[0.4245, 0.4245, 0.4245, 0.4245, 0.4245, 0.4245, 0.4245, 0.4245,
           0.4245, 0.4245, 0.4245, 0.4245, 0.4245, 0.4245, 0.4245, 0.4245,
           0.4245, 0.4245, 0.4245, 0.4245, 0.4245, 0.4245, 0.4245, 0.4245,
           0.4245, 0.4245, 0.4245, 0.4245, 0.4245, 0.4245, 0.4245, 0.4245,
           0.4245, 0.4245, 0.4245, 0.4245],
          [0.9751, 0.9751, 0.9751, 0.9751, 0.9751, 0.9751, 0.9751, 0.9751,
           0.9751, 0.9751, 0.9751, 0.9751, 0.9751, 0.9751, 0.9751, 0.9751,
           0.9751, 0.9751, 0.9751, 0.9751, 0.9751, 0.9751, 0.9751, 0.9751,
           0.9751, 0.9751, 0.9751, 0.9751, 0.9751, 0.9751, 0.9751, 0.9751,
           0.9751, 0.9751, 0.9751, 0.9751],
          [1.0531, 1.0531, 1.0531, 1.0531, 1.0531, 1.0531, 1.0531, 1.0531,
           1.0531, 1.0531, 1.0531, 1.0531, 1.0531, 1.0531, 1.0531, 1.0531,
           1.0531, 1.0531, 1.0531, 1.0531, 1.0531, 1.0531, 1.0531, 1.0531,
           1.0531, 1.0531, 1.0531, 1.0531, 1.0531, 1.0531, 1.0

## Loss calculation

Now we have to see how good our ~shit~ prediction is.  Since we haven't done training, and we saw that regardless of example we had the same exact logit values, we can expect it's bad. That said, we need to know how bad. For this example we'll use cross entropy, also known as the negative log likelihood of the softmax.  Our loss calculates

$$
\ell_i=-\log\big(\mathrm{softmax}(z_i)\_{y_i}\big)
= -z_{i,y_i}+\log\!\sum_{c=1}^C e^{z_{i,c}},
$$


To calculate loss we'll pass in the calculated `logits` and our next tokens stored in `y`. The cross entropy function does not respect batches so we'll flatten the `B` dimension for both `logits` and `y`

In [51]:
import torch.nn.functional as F

In [52]:
y_flat = y.view(-1)
y_flat.shape, y_flat

(torch.Size([16]),
 tensor([15, 32,  9,  5, 20, 30, 15, 11,  9,  6, 20,  5,  0, 13, 21,  0]))

In [53]:
logits_flat = logits.view(-1, logits.size(-1))
logits_flat.shape, logits_flat

(torch.Size([16, 36]),
 tensor([[0.4245, 0.4245, 0.4245, 0.4245, 0.4245, 0.4245, 0.4245, 0.4245, 0.4245,
          0.4245, 0.4245, 0.4245, 0.4245, 0.4245, 0.4245, 0.4245, 0.4245, 0.4245,
          0.4245, 0.4245, 0.4245, 0.4245, 0.4245, 0.4245, 0.4245, 0.4245, 0.4245,
          0.4245, 0.4245, 0.4245, 0.4245, 0.4245, 0.4245, 0.4245, 0.4245, 0.4245],
         [0.9751, 0.9751, 0.9751, 0.9751, 0.9751, 0.9751, 0.9751, 0.9751, 0.9751,
          0.9751, 0.9751, 0.9751, 0.9751, 0.9751, 0.9751, 0.9751, 0.9751, 0.9751,
          0.9751, 0.9751, 0.9751, 0.9751, 0.9751, 0.9751, 0.9751, 0.9751, 0.9751,
          0.9751, 0.9751, 0.9751, 0.9751, 0.9751, 0.9751, 0.9751, 0.9751, 0.9751],
         [1.0531, 1.0531, 1.0531, 1.0531, 1.0531, 1.0531, 1.0531, 1.0531, 1.0531,
          1.0531, 1.0531, 1.0531, 1.0531, 1.0531, 1.0531, 1.0531, 1.0531, 1.0531,
          1.0531, 1.0531, 1.0531, 1.0531, 1.0531, 1.0531, 1.0531, 1.0531, 1.0531,
          1.0531, 1.0531, 1.0531, 1.0531, 1.0531, 1.0531, 1.0531, 1.0531,

In [54]:
loss = F.cross_entropy(logits_flat, y_flat)
loss.shape, loss

(torch.Size([]), tensor(3.5835, grad_fn=<NllLossBackward0>))

## Back Propagation

We now know just how ~well~ terribly the current model, with its weights and biases, predicts the next token given the input context. We now need to know how to change the different weights and biases to improve the formula.  We could do this by guessing through making minor changes and seeing what improves, or we can think through this more critically.

If you review the chain of layers above, you can see that it's a series of formulas.  We can think of this as $f(g(x))$, except with many many more layers and complexities.  Since this is a formula, we can dig into our math toolbox and find a better way to determine what parts need to update.  Recall that in our calculus we learned that differentiation tells us the rate of change in a graph.  So if we treat the loss function $\mathcal{L}$ as $\mathcal{L}(f(g(x)))$ taking the partial differential 

$\delta=\partial \mathcal{L}/\partial h$

at each layer will give us the impact of each weight/bias on our final out (albeit the inverse since our loss function is the negative log likelihood). 

Lucky for us, each layer of our model already has a placeholder for the partial differential called the **Gradient**. We'll use this field to store it.  We'll start by first zeroing out the gradients. We do this because of the nature of handling partial differentials for multiple dependencies. Recall that in multiple places we had a formula structure of 

$a+b=c ; a+c= d$

In this case $a$ has 2 dependencies and determining the partial derivative of $\partial d / \partial a$ requires understanding both the path from $d$ and $c$.  To determine the true impact of a we would sum both partial derivatives together.  Because of this property, the tool we use, the built in `.backwards()` automatically sums gradients, `+=`, so if we do not set the gradient to `0` we then end up with erroneous gradients. 

Finally, we start `.backwards` from the `loss`, not `logits` as our goal is to minimize loss, we need to ensure we are looking at the calculations that impact loss which requires the whole forward pass to be able to generate the prediction `logits_flat`.  If we think of it as $\mathcal{L}(f(x))$ where $f(x)$ is the forward pass to generate logits, then a simple chain rule is applied:

${\partial}/{\partial x} =  \mathcal{L}'(f(x)) f'(x)$

Lets start by zeroing the gradients and leaning on pytorch to calculate the gradients for us. We'll also validate the gradients were `none`.

In [55]:
lm_head.zero_grad()
h2g.zero_grad()
x2g.zero_grad()
wte.zero_grad()


# validate gradients
lm_head.weight.grad, wte.weight.grad

(None, None)

**Auto-Diff** - Now let's see the magic of the gradients populate.  This magic is called auto-differentiation, or auto-diff for short. This allows us to not have to write many layers of nasty code to do the differentiation for us, but, if you're a sadist, you can surely find people who have written out that code (it's not too bad since you just do one layer at a time). 

In [56]:
loss.backward()

In [57]:
lm_head.weight.grad, wte.weight.grad

(tensor([[-0.0577, -0.0584, -0.0583, -0.0606],
         [ 0.0119,  0.0112,  0.0113,  0.0090],
         [ 0.0119,  0.0112,  0.0113,  0.0090],
         [ 0.0119,  0.0112,  0.0113,  0.0090],
         [ 0.0119,  0.0112,  0.0113,  0.0090],
         [-0.0388, -0.0141, -0.0394, -0.0417],
         [-0.0033, -0.0040, -0.0039, -0.0062],
         [ 0.0119,  0.0112,  0.0113,  0.0090],
         [ 0.0119,  0.0112,  0.0113,  0.0090],
         [-0.0203, -0.0210,  0.0010, -0.0232],
         [ 0.0119,  0.0112,  0.0113,  0.0090],
         [-0.0289, -0.0296, -0.0295,  0.0090],
         [ 0.0119,  0.0112,  0.0113,  0.0090],
         [-0.0223, -0.0230, -0.0229,  0.0090],
         [ 0.0119,  0.0112,  0.0113,  0.0090],
         [-0.0338, -0.0345, -0.0344, -0.0367],
         [ 0.0119,  0.0112,  0.0113,  0.0090],
         [ 0.0119,  0.0112,  0.0113,  0.0090],
         [ 0.0119,  0.0112,  0.0113,  0.0090],
         [ 0.0119,  0.0112,  0.0113,  0.0090],
         [-0.0402, -0.0409, -0.0408, -0.0129],
         [-0.

## Learning

The process of learning now requires us to update our weights based on this gradient. To really feel the "back propagation" we'll start with the last layer and work backwards, though, since we have all of the gradients calculated already, the order does not matter. Recall that our loss function is the negative log likelihood ratio so our gradient signs are flipped.  If a parameter is important, the gradient will be more negative, and vice versa. The gradients are a ratio of importance of each parameter and we need to know how much of that gradient to apply to our weights. This "how much" is referred to as the *learning rate*. In modern training learning rate schedulers and optimizers are used to vary the rate and application by layer and by training round with learning rates that are small (e.g. 1e-3) and decaying. 

We however are trying to learn and if you look at the gradient above in most layers it's tiny (~1e-10).  If we used a typical learning rate scheduler, with our context, batch size, and just 1 pass, the second pass would just have the same values and we wouldn't learn anything new.  Because of this we'll do something super unorthodox.  Our LM_head will use a learning rate of `5.000` but for the rest of the layers we'll use a learning rate of `1e6`. This INSANE learning rate would absolutely be too noisy for real training as taking that "large" of steps would mean the model could never find a good fit.  But since we're doing 1 step we don't care. Also, we're rerunning on the same example we used in the first loop so even less of a care. But as a warning, DO NOT DO THIS IN REAL TRAINING. If you did your model would most likely not converge. 

In [58]:
## Huge learning rate to emphasize
head_learning_rate = 5.000
learning_rate = 1e6

### Output Layer
Let's start with our output layer.  Recall that we initialized the weights to `1.000` so we can quickly see the impact of the gradient update on the weights. Most notable that we'll see is that the entries corresponding with the token that are present in `y`, our target, are up weighted and others are downweighted. This should already intuitively give you a sense as to 
1. The model will be improved
2. This is WAY too high of a learning rate if we wanted to generalize.

Additionally you'll notice that the gradient is differs across the `n_embd` dimensions. This shows that even though we have done only a single pass, because we have multiple examples our model is already learning deeper structures. Finally, since this is the output layer, we did not include any bias 

In [59]:
with torch.no_grad():
    lm_head.weight -= head_learning_rate * lm_head.weight.grad
lm_head.weight

Parameter containing:
tensor([[1.2886, 1.2921, 1.2917, 1.3032],
        [0.9404, 0.9439, 0.9434, 0.9550],
        [0.9404, 0.9439, 0.9434, 0.9550],
        [0.9404, 0.9439, 0.9434, 0.9550],
        [0.9404, 0.9439, 0.9434, 0.9550],
        [1.1939, 1.0706, 1.1969, 1.2085],
        [1.0165, 1.0200, 1.0196, 1.0311],
        [0.9404, 0.9439, 0.9434, 0.9550],
        [0.9404, 0.9439, 0.9434, 0.9550],
        [1.1014, 1.1050, 0.9948, 1.1160],
        [0.9404, 0.9439, 0.9434, 0.9550],
        [1.1443, 1.1478, 1.1474, 0.9550],
        [0.9404, 0.9439, 0.9434, 0.9550],
        [1.1114, 1.1149, 1.1144, 0.9550],
        [0.9404, 0.9439, 0.9434, 0.9550],
        [1.1688, 1.1723, 1.1718, 1.1834],
        [0.9404, 0.9439, 0.9434, 0.9550],
        [0.9404, 0.9439, 0.9434, 0.9550],
        [0.9404, 0.9439, 0.9434, 0.9550],
        [0.9404, 0.9439, 0.9434, 0.9550],
        [1.2009, 1.2044, 1.2039, 1.0646],
        [1.1286, 1.1321, 1.1316, 1.1432],
        [0.9404, 0.9439, 0.9434, 0.9550],
        [0.9

### Recurrent Unit - GRU
Next we'll update our two linear layers inside of the GRU.  Recall that we initiated h2g to weights `0.2500`, x2g weights to `0.5000` and the biases to.  Our gradients on this layer are extremely small (~1e-9). We can see that the gradients are both positive and negative suggesting the model is starting to learn patterns in the position of text that impact the next token prediction.  Also we can see that `x2g` has consistent gradients across all embedding positions, but `h2g` does not, showing that `x2g` is starting to learn more complex embedding relationships. Together, this is starting to show how the different embedding layers can impact the recursion. 

With these layers you can see the beauty of our insanely high learning rate.  Even though our gradient impact was tiny, we can see the weight shift away from the initialization suggesting some level of adaptation. 

In [60]:
with torch.no_grad():
    h2g.weight -= learning_rate * h2g.weight.grad
    h2g.bias -= learning_rate * h2g.bias.grad
h2g.weight, h2g.bias

(Parameter containing:
 tensor([[0.2606, 0.2606, 0.2606, 0.2606],
         [0.2599, 0.2599, 0.2599, 0.2599],
         [0.2602, 0.2602, 0.2602, 0.2602],
         [0.2582, 0.2582, 0.2582, 0.2582],
         [0.2499, 0.2499, 0.2499, 0.2499],
         [0.2499, 0.2499, 0.2499, 0.2499],
         [0.2499, 0.2499, 0.2499, 0.2499],
         [0.2500, 0.2500, 0.2500, 0.2500],
         [0.2486, 0.2486, 0.2486, 0.2486],
         [0.2487, 0.2487, 0.2487, 0.2487],
         [0.2487, 0.2487, 0.2487, 0.2487],
         [0.2490, 0.2490, 0.2490, 0.2490]], requires_grad=True),
 Parameter containing:
 tensor([ 0.0538,  0.0488,  0.0503,  0.0440, -0.0002, -0.0002, -0.0002, -0.0001,
         -0.0063, -0.0058, -0.0059, -0.0051], requires_grad=True))

In [61]:
with torch.no_grad():
    x2g.weight -= learning_rate * x2g.weight.grad
    x2g.bias -= learning_rate * x2g.bias.grad
x2g.weight, x2g.bias

(Parameter containing:
 tensor([[0.5539, 0.5322, 0.5504, 0.5576],
         [0.5484, 0.5285, 0.5464, 0.5521],
         [0.5501, 0.5299, 0.5478, 0.5538],
         [0.5454, 0.5250, 0.5415, 0.5474],
         [0.4999, 0.4999, 0.4999, 0.4998],
         [0.4999, 0.4999, 0.4999, 0.4998],
         [0.4999, 0.4999, 0.4999, 0.4998],
         [0.4999, 0.4999, 0.4999, 0.4999],
         [0.4936, 0.4962, 0.4941, 0.4922],
         [0.4942, 0.4966, 0.4945, 0.4928],
         [0.4941, 0.4965, 0.4944, 0.4927],
         [0.4946, 0.4972, 0.4951, 0.4937]], requires_grad=True),
 Parameter containing:
 tensor([ 0.0538,  0.0488,  0.0503,  0.0440, -0.0002, -0.0002, -0.0002, -0.0001,
         -0.0073, -0.0067, -0.0068, -0.0059], requires_grad=True))

### Input Layer
Finally we'll update our input layer. Recall that we initialized it to `1.000`. Given our high learning rate, a keen eye can observe an interesting phenomenon, gradient updates only impacted the tokens that were used in our example. This differs from our other layers because our embedding layer projects the example into the embedding space only uses vectors for tokens present so those are the only ones that the gradient can trace back to, versus on other layers because of dot products and other operations there's impact across the whole tensor. 

Also notice that the gradient is different for the embedding channel, once again suggesting that our model is starting to learn deeper patterns in the data. 

In [62]:
with torch.no_grad():
    wte.weight -= learning_rate * wte.weight.grad
wte.weight

Parameter containing:
tensor([[1.0034, 1.0034, 1.0034, 1.0000],
        [1.0000, 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000, 1.0000],
        [1.0061, 1.0037, 1.0098, 1.0098],
        [1.0103, 1.0000, 1.0103, 1.0103],
        [1.0000, 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000, 1.0000],
        [1.0131, 1.0131, 1.0131, 1.0131],
        [1.0000, 1.0000, 1.0000, 1.0000],
        [1.0163, 1.0000, 1.0163, 1.0163],
        [1.0000, 1.0000, 1.0000, 1.0000],
        [1.0026, 1.0026, 1.0000, 1.0026],
        [1.0000, 1.0000, 1.0000, 1.0000],
        [1.0112, 1.0004, 1.0112, 1.0112],
        [1.0000, 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000, 1.0000],
        [1.0060, 1.0081, 1.0060, 1.0081],
        [1.0012, 1.0012, 1.0012, 1.0012],
        [1.0000, 1.0000, 1.0000, 1.0000],
        [1.0

## Forward Pass with Updated Weights

Now that we have the updated weights for each layer, let's do another forward pass and compare the loss. Since each layer was previously explained we will instead focus on just showing the outputs of the different layers and the final loss. If you want, you can check the previous outputs in the cached cell outputs above and compare them to see how the weight changes impacted the values at each layer. 

You'll notice that because our high learning rate we're able to see how each layer now shifts the embedding values as the input passes through them. 

### Data Re-loading
Repulling to a new `x_2`. We'll keep `y` to emphasize the same examples are being used. 

In [63]:
x_2 = tok_for_training[:-1].view(B, T)
x_2, y

(tensor([[35, 15, 32,  9,  5, 20, 30, 15],
         [11,  9,  6, 20,  5,  0, 13, 21]]),
 tensor([[15, 32,  9,  5, 20, 30, 15, 11],
         [ 9,  6, 20,  5,  0, 13, 21,  0]]))

### Input Layer
Note that in `wte` since the gradient impact was only on the embeddings that mapped to our tokens. Lucky for us, we're passing the same example through so we'll pull those same weights.

With RNNs, because of the recursion layers, we would need long training and long context to see changes in our input layer initiation. Alternatively, a common step to help improve the input embedding is to do weight-tying between the input and output layers. This ensures that the input and output layers embed in the same space and helps with learning stabilization, but weight tying goes in and out of flavor often.  Because of this we'll cover weight-tying in the GPT notebook I write. 

In [64]:
x = wte(x_2)
x.shape, x 

(torch.Size([2, 8, 4]),
 tensor([[[1.0094, 1.0094, 1.0094, 1.0094],
          [1.0112, 1.0004, 1.0112, 1.0112],
          [1.0074, 1.0074, 1.0000, 1.0074],
          [1.0131, 1.0131, 1.0131, 1.0131],
          [1.0061, 1.0037, 1.0098, 1.0098],
          [1.0060, 1.0081, 1.0060, 1.0081],
          [1.0000, 1.0014, 1.0014, 1.0014],
          [1.0112, 1.0004, 1.0112, 1.0112]],
 
         [[1.0163, 1.0000, 1.0163, 1.0163],
          [1.0131, 1.0131, 1.0131, 1.0131],
          [1.0103, 1.0000, 1.0103, 1.0103],
          [1.0060, 1.0081, 1.0060, 1.0081],
          [1.0061, 1.0037, 1.0098, 1.0098],
          [1.0034, 1.0034, 1.0034, 1.0000],
          [1.0026, 1.0026, 1.0000, 1.0026],
          [1.0012, 1.0012, 1.0012, 1.0012]]], grad_fn=<EmbeddingBackward0>))

### Dropout 
Once again we'll perform dropout. Dropout is a stateless layer, meaning it does not have any learnable parameters, so we have a consistent, though still random, outcome. 

In [65]:
x = dropout(x)
x

tensor([[[1.1216, 1.1216, 1.1216, 1.1216],
         [0.0000, 1.1115, 1.1235, 1.1235],
         [1.1193, 1.1193, 1.1111, 1.1193],
         [1.1256, 1.1256, 1.1256, 1.1256],
         [1.1179, 1.1152, 1.1220, 1.1220],
         [1.1178, 1.1202, 1.1178, 1.1202],
         [1.1111, 1.1127, 1.1127, 1.1127],
         [1.1235, 1.1115, 1.1235, 1.1235]],

        [[0.0000, 1.1111, 1.1292, 1.1292],
         [1.1256, 0.0000, 1.1256, 1.1256],
         [1.1225, 1.1111, 1.1225, 1.1225],
         [1.1178, 1.1202, 1.1178, 1.1202],
         [1.1179, 1.1152, 1.1220, 0.0000],
         [1.1149, 1.1149, 1.1149, 1.1111],
         [1.1141, 1.1141, 1.1111, 1.1141],
         [1.1125, 1.1125, 1.1125, 1.1125]]], grad_fn=<MulBackward0>)

### Recurrent Unit - GRU

This time through we'll run the recurrent layer for all time periods and not break out a specific time period.  We still have to reset our hidden layer to `0` since our training batch reset. Given the high learning rate we will now see how different weights and biases impact our convolution. 

In [66]:
h_t = torch.zeros(B_batch, hidden_size) 
hs = x.new_empty(B_batch, T_context, hidden_size)

In [67]:
for t in range(T_context):
    
    # set input and previous
    h_prev = h_t
    x_t = x[:, t, :]

    # calculate weights
    hz, hr, hh = h2g(h_prev).split(hidden_size, dim=-1)
    xz, xr, xh = x2g(x_t).split(hidden_size, dim=-1)

    # update gate
    z = torch.sigmoid(xz + hz)

    # reset gate
    r = torch.sigmoid(xr + hr)

    # candidate hidden state
    h_cand = torch.tanh(xh + (r * hh))

    # recursion output
    h_cand_imp = (1 - z) * h_cand
    h_prev_imp = z * h_prev
    h_t = h_cand_imp + h_prev_imp

    # save h_t
    hs[:, t, :] = h_t   

hs

tensor([[[0.0695, 0.0715, 0.0709, 0.0734],
         [0.1712, 0.1753, 0.1739, 0.1792],
         [0.2203, 0.2256, 0.2239, 0.2307],
         [0.2636, 0.2699, 0.2679, 0.2759],
         [0.3033, 0.3104, 0.3081, 0.3172],
         [0.3394, 0.3473, 0.3447, 0.3547],
         [0.3728, 0.3814, 0.3786, 0.3894],
         [0.4031, 0.4121, 0.4091, 0.4206]],

        [[0.1157, 0.1183, 0.1175, 0.1209],
         [0.2058, 0.2103, 0.2088, 0.2146],
         [0.2510, 0.2567, 0.2549, 0.2621],
         [0.2919, 0.2985, 0.2964, 0.3047],
         [0.3554, 0.3629, 0.3605, 0.3699],
         [0.3875, 0.3955, 0.3929, 0.4031],
         [0.4170, 0.4256, 0.4228, 0.4336],
         [0.4443, 0.4533, 0.4504, 0.4618]]], grad_fn=<CopySlices>)

### Final Dropout
As with before, we'll run dropout again before running the model head.  This stateless layer will again perform similar to the last time we ran it, just with random dropout. 

In [68]:
x = dropout(x)
x

tensor([[[1.2462, 1.2462, 1.2462, 1.2462],
         [0.0000, 1.2350, 1.2484, 1.2484],
         [1.2437, 1.2437, 1.2346, 0.0000],
         [1.2507, 1.2507, 1.2507, 1.2507],
         [1.2421, 1.2392, 1.2467, 1.2467],
         [1.2420, 1.2446, 1.2420, 1.2446],
         [1.2346, 0.0000, 1.2363, 1.2363],
         [1.2484, 1.2350, 1.2484, 0.0000]],

        [[0.0000, 1.2346, 1.2546, 1.2546],
         [1.2507, 0.0000, 1.2507, 1.2507],
         [0.0000, 1.2346, 1.2473, 1.2473],
         [1.2420, 1.2446, 1.2420, 1.2446],
         [1.2421, 0.0000, 1.2467, 0.0000],
         [1.2388, 1.2388, 1.2388, 1.2346],
         [1.2378, 1.2378, 1.2346, 1.2378],
         [1.2361, 1.2361, 1.2361, 1.2361]]], grad_fn=<MulBackward0>)

### Output Layers AKA Model Head.
Now we do our final projection against the LM head. We have seen our Y and, if we select some examples, we can see that the logit for the  examples are upweighted (e.g. position 15 on example 1, position 32 on example 2). We can also see that this training is not perfect since, if we look at those positions in other examples (e.g. position 32 in example 1) it's also upweighted.  This shows that while our model is learning, it still has a long way to go before being trained. 

In [69]:
logits = lm_head(x)
logits.shape, logits

(torch.Size([2, 8, 36]),
 tensor([[[6.4500, 4.7140, 4.7140, 4.7140, 4.7140, 5.8197, 5.0935, 4.7140,
           4.7140, 5.3802, 4.7140, 5.4765, 4.7140, 5.3533, 4.7140, 5.8527,
           4.7140, 4.7140, 4.7140, 4.7140, 5.8246, 5.6521, 4.7140, 4.7140,
           4.7140, 4.7140, 4.7140, 4.7140, 4.7140, 4.7140, 5.6074, 4.7140,
           5.0938, 4.7140, 4.7140, 4.7140],
          [4.8352, 3.5357, 3.5357, 3.5357, 3.5357, 4.3250, 3.8197, 3.5357,
           3.5357, 3.9998, 3.5357, 4.0421, 3.5357, 3.9603, 3.5357, 4.3880,
           3.5357, 3.5357, 3.5357, 3.5357, 4.3195, 4.2379, 3.5357, 3.5357,
           3.5357, 3.5357, 3.5357, 3.5357, 3.5357, 3.5357, 4.2044, 3.5357,
           3.8200, 3.5357, 3.5357, 3.5357],
          [4.8043, 3.5081, 3.5081, 3.5081, 3.5081, 4.2940, 3.7915, 3.5081,
           3.5081, 3.9722, 3.5081, 4.2672, 3.5081, 4.1446, 3.5081, 4.3583,
           3.5081, 3.5081, 3.5081, 3.5081, 4.4777, 4.2085, 3.5081, 3.5081,
           3.5081, 3.5081, 3.5081, 3.5081, 3.5081, 3.5081, 4.1

### Updated Loss calculation

Now we'll calculate the updated loss.  Our first pass's loss was 3.5835, on par with random. Since we're passing through the same example and used a fairly high learning rate we should see a significant improvement with just 1 learning pass. 

In [70]:
loss

tensor(3.5835, grad_fn=<NllLossBackward0>)

In [71]:
y_flat = y.view(-1)
logits_flat = logits.view(-1, logits.size(-1))
updated_loss = F.cross_entropy(logits_flat, y_flat)
print(updated_loss.shape, updated_loss)

torch.Size([]) tensor(3.0763, grad_fn=<NllLossBackward0>)


In [72]:
f'1 round of training resulted in an loss improvment of {loss.item() - updated_loss.item():.4f}'

'1 round of training resulted in an loss improvment of 0.5072'

# SUCCESS!
Our training improved the loss by about **~13%** (amount may vary since we didn't set a seed). There are flaws with this, mainly passing the same example through a second time and a high learning rate, but this helps show the fundamentals of what learning does inside a GRU style RNN model. 